In [1]:
import cv2
import shutil
import yt_dlp
import time
import os
from datetime import datetime
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from ultralytics import YOLO

# ==============================================================================
# 1. CẤU HÌNH HỆ THỐNG
# ==============================================================================
def setup_system(base_path_str):
    base_dir = Path(base_path_str)
    
    paths = {
        "check":      base_dir / "1_Check",
        "helmet":     base_dir / "2_Detect_Helmet",
        "no_helmet":  base_dir / "3_Detect_No_Helmet",
        "no_detect":  base_dir / "5_No Detect",
        "archive":    base_dir / "9_Archieve"
    }
    for p in paths.values():
        p.mkdir(parents=True, exist_ok=True)
    return base_dir, paths

def save_image_async(file_path, frame):
    try:
        cv2.imwrite(str(file_path), frame)
        print(f"   [Async] 💾 Đã lưu: {file_path.name}")
    except Exception as e:
        print(f"   ❌ Lỗi lưu: {e}")

# ==============================================================================
# 2. GIAI ĐOẠN 1: CẮT ẢNH (BẢN ỔN ĐỊNH - CHỐNG LỖI HLS)
# ==============================================================================
def capture_youtube_to_folder(paths, youtube_url, cam_id="Cam_1", interval_sec=10):
    print("\n" + "="*40)
    print("🎬 GIAI ĐOẠN 1: KẾT NỐI STREAM (STABLE MODE)...")
    print("="*40)

    # CẤU HÌNH QUAN TRỌNG: Format 18 là MP4 360p (Rất nhẹ, link trực tiếp, không phải HLS)
    # Hoặc lấy best mp4 dưới 720p để tránh link manifest
    ydl_opts = {
        'format': '18/best[ext=mp4][height<=720]', 
        'quiet': True,
        'no_warnings': True,
        'force_url': True 
    }
    
    video_url = None
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print("⏳ Đang giải mã link YouTube (Xin chờ)...")
            info = ydl.extract_info(youtube_url, download=False)
            video_url = info['url']
            print(f"✅ Đã lấy link: {info.get('title', 'Video')}")
            # Kiểm tra xem có phải link manifest không (nguyên nhân gây lỗi)
            if "manifest" in video_url:
                print("⚠️ Cảnh báo: Vẫn là link Manifest. Hệ thống sẽ cố gắng xử lý...")
    except Exception as e:
        print(f"❌ Lỗi lấy link: {e}")
        return False

    cap = cv2.VideoCapture(video_url)
    
    # Kiểm tra kết nối
    if not cap.isOpened():
        print("❌ OpenCV không mở được link này. Thử lại sau.")
        return False

    # Đọc thử 1 frame đầu tiên để chắc chắn stream sống
    ret, test_frame = cap.read()
    if not ret:
        print("❌ Kết nối được nhưng không đọc được frame đầu tiên. Stream có thể đã tắt.")
        return False
    else:
        print("✅ Stream hoạt động tốt! Bắt đầu cắt...")

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0 or fps > 120: fps = 30
    frame_step = int(fps * interval_sec)
    
    executor = ThreadPoolExecutor(max_workers=2)
    frame_count = 0
    saved_count = 0
    retry_count = 0 # Đếm số lần thử lại nếu mạng rớt
    
    try:
        while True:
            # ĐỌC TUẦN TỰ (Chậm hơn chút nhưng CHẮC CHẮN ĐƯỢC)
            ret, frame = cap.read()
            
            if not ret:
                retry_count += 1
                if retry_count > 10: # Nếu lỗi liên tiếp 10 lần mới thoát
                    print("⚠️ Mất kết nối Stream quá lâu. Dừng cắt.")
                    break
                time.sleep(0.1) # Đợi chút rồi thử lại
                continue
            
            # Reset retry nếu đọc thành công
            retry_count = 0

            # Logic: Chỉ lưu khi chia hết cho frame_step
            if frame_count % frame_step == 0:
                now_str = datetime.now().strftime("%d%m%Y_%H%M%S")
                # Thêm số frame_count vào tên để tránh trùng lặp nếu cắt nhanh
                file_name = f"{cam_id}_PPE_{now_str}_{frame_count}.jpg"
                file_path = paths["check"] / file_name
                
                # Lưu async
                executor.submit(save_image_async, file_path, frame.copy())
                saved_count += 1
            
            frame_count += 1

            # DEMO: Chỉ cắt 10 ảnh rồi dừng để chuyển sang AI (Bạn có thể bỏ dòng này)
            if saved_count >= 10: 
                print("🛑 Đã đủ 10 ảnh demo. Dừng để chuyển sang bước AI.")
                break

    except KeyboardInterrupt:
        print("\n⏹️ Dừng thủ công.")
    finally:
        cap.release()
        executor.shutdown(wait=True)
        print(f"\n✅ KẾT THÚC GIAI ĐOẠN 1. Tổng ảnh: {saved_count}")
        
    return saved_count > 0

# ==============================================================================
# 3. GIAI ĐOẠN 2: AI CHECK (GIỮ NGUYÊN CODE ĐÚNG CỦA BẠN)
# ==============================================================================
def process_check_folder(base_dir, paths):
    print("\n" + "="*40)
    print("🧠 GIAI ĐOẠN 2: AI PHÂN TÍCH...")
    print("="*40)

    model_path = base_dir / "best.pt"
    if not model_path.exists():
        print("❌ Không tìm thấy best.pt")
        return

    model = YOLO(str(model_path))
    image_files = list(paths["check"].glob('*.jpg')) + list(paths["check"].glob('*.png'))
    
    if not image_files:
        print("⚠️ Không có ảnh trong folder Check.")
        return

    print(f"🚀 Xử lý {len(image_files)} ảnh...")
    
    # Từ khóa
    KEYWORDS_SAFE = ['helmet', 'hardhat', 'safety-cap']
    KEYWORDS_UNSAFE = ['no-helmet', 'head', 'face', 'person', 'no_helmet']

    results = model.predict(source=image_files, conf=0.1, iou=0.3, verbose=False)

    for result in results:
        origin_file = Path(result.path)
        filename = origin_file.name
        img = cv2.imread(str(origin_file))
        if img is None: continue

        target_folder = paths["no_detect"]
        has_violation = False
        has_safe = False
        
        for box in result.boxes:
            cls_name = model.names[int(box.cls[0])].lower().replace("_", "-")
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            is_unsafe = any(kw in cls_name for kw in KEYWORDS_UNSAFE)
            if is_unsafe and conf >= 0.1:
                has_violation = True
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                cv2.putText(img, f"NO-HELMET {conf:.2f}", (x1, y1-10), 0, 0.6, (0,0,255), 2)
            
            elif any(kw in cls_name for kw in KEYWORDS_SAFE) and conf >= 0.4:
                has_safe = True
                if not has_violation:
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(img, f"Helmet {conf:.2f}", (x1, y1-10), 0, 0.5, (0,255,0), 1)

        if has_violation: target_folder = paths["no_helmet"]
        elif has_safe: target_folder = paths["helmet"]

        cv2.imwrite(str(target_folder / filename), img)
        try:
            shutil.move(str(origin_file), str(paths["archive"] / filename))
        except: pass
        
        status = "❌ VI PHẠM" if has_violation else "✅ AN TOÀN"
        print(f"   {status} | {filename}")

    print("\n🎉 HOÀN TẤT!")

# ==============================================================================
# MAIN
# ==============================================================================
if __name__ == "__main__":
    WORK_PATH = r"D:\THS\06.1 Python\AI Camera"
    #LINK = "https://www.youtube.com/watch?v=LCauiIO0Bt4"
    LINK = "https://www.youtube.com/watch?v=5BVLM3RZS2Y"
    
    BASE, PATHS = setup_system(WORK_PATH)
    
    # Chạy Giai đoạn 1 (Chắc chắn cắt được ảnh)
    if capture_youtube_to_folder(PATHS, LINK, interval_sec=10):
        # Chạy Giai đoạn 2
        process_check_folder(BASE, PATHS)
    else:
        print("⚠️ Kiểm tra lại link YouTube.")


🎬 GIAI ĐOẠN 1: KẾT NỐI STREAM (STABLE MODE)...
⏳ Đang giải mã link YouTube (Xin chờ)...
✅ Đã lấy link: Saudi Arab Riyadh Randa company
✅ Stream hoạt động tốt! Bắt đầu cắt...
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101938_0.jpg
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101939_300.jpg
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101940_600.jpg
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101942_900.jpg
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101943_1200.jpg
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101944_1500.jpg
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101945_1800.jpg
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101947_2100.jpg
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101947_2400.jpg
🛑 Đã đủ 10 ảnh demo. Dừng để chuyển sang bước AI.
   [Async] 💾 Đã lưu: Cam_1_PPE_31122025_101949_2700.jpg

✅ KẾT THÚC GIAI ĐOẠN 1. Tổng ảnh: 10

🧠 GIAI ĐOẠN 2: AI PHÂN TÍCH...
🚀 Xử lý 10 ảnh...
   ❌ VI PHẠM | Cam_1_PPE_31122025_101938_0.jpg
   ❌ VI PHẠM | Cam_1_PPE_31122025_101939_300.jpg
   ✅ AN TOÀN | Cam_1_PPE_31122025_101